<a href="https://colab.research.google.com/github/asking28/finsummary/blob/master/combining_data_word2vec_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
import pandas as pd
import gensim
import glob
import nltk.data
from tqdm import tqdm
import ast
import re
from tqdm import tqdm

paths=['/content/drive/My Drive/finsummary/Data/validation/annual_reports','/content/drive/My Drive/finsummary/Data/testing/annual_reports','/content/drive/My Drive/finsummary/Data/training/annual_reports']
word_tokenizer=nltk.word_tokenize

# glob.glob('/content/drive/My Drive/finsummary/Data/validation/annual_reports/*.txt')

nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# fp = open('/content/drive/My Drive/finsummary/Data/validation/annual_reports/31201.txt')
# data = fp.read()
# for sent in tokenizer.tokenize(data):
  # sent.replace('\n',' ')
# print('\n-----\n'.join(tokenizer.tokenize(data)))
# sentences=[]
# for dir_p in paths:
#   for i,file_p in enumerate(glob.glob(dir_p+'/*.txt')):
#     fp=open(file_p)
#     data=fp.read()
#     for sent in tokenizer.tokenize(data):
#       sent=sent.replace('\n',' ')
#       sent=word_tokenizer(sent)
#       sentences.append(sent)
#     if i%100==0:
#       df=pd.DataFrame({'Sentences':sentences})
#       df.to_csv(dir_p+'/combine_sents_emb_train'+str(i)+'.csv')
#       sentences=[]
#   df=pd.DataFrame({'Sentences':sentences})
#   df.to_csv(dir_p+'/combine_sents_emb_train'+str(i)+'.csv')
#   print(dir_p)
# df=pd.DataFrame({'Sentences':sentences})
# df.to_csv('/content/drive/My Drive/finsummary/combine_sents_emb_train.csv')

sentences=[]
for dir_p in paths:
  for i,file_p in enumerate(glob.glob(dir_p+'/*.csv')):
    df=pd.read_csv(file_p,converters={'Sentences':ast.literal_eval})
    for i in tqdm(range(len(df))):
      sentences.append(['<s>']+df['Sentences'][i]+[r'<\s>'])

In [ ]:
# df=pd.read_csv('/content/drive/My Drive/finsummary/word2vec/combine_sents_emb_train.csv')

In [ ]:
# df.shape

In [ ]:
# df.head()

In [ ]:
# import nltk
# nltk.download('punkt')
# tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
def remove_nn(a):
  result = re.sub('[^0-9a-zA-Z]',' ', a)
  return result

In [ ]:
for i in tqdm(range(len(sentences))):
  sentences[i]=remove_nn(sentences[i])

In [ ]:
model=gensim.models.Word2Vec(size=300,min_count=3,workers=16,sg=1)
model.build_vocab(sentences)
# model.train(sentences,total_examples=model.corpus_count,epochs=model.iter)
# save_dir='/content/drive/My Drive/finsummary/Data'


In [ ]:
model.train(sentences,total_examples=model.corpus_count,epochs=model.iter)

In [ ]:
model.save(os.path.join(save_dir, 'word2vec.{}d.{}k.bin'.format(300, len(model.wv.vocab)//1000)))
model.wv.save_word2vec_format(os.path.join(save_dir,'word2vec.{}d.{}k.w2v'.format(300, len(model.wv.vocab)//1000)))